# Project 3 Part 2 (Core)

Rodrigo Arguello-Serrano

In [3]:
#Import Modules
import pandas as pd
import numpy as np
import os, time, json

import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook

FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.DS_Store',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'title_akas.csv',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz',
 'tmdb_results_combined.csv.gz',
 'title_ratings.csv.gz']

In [4]:
import json
with open('/Users/rodrigoarguelloserrano/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

#import the package and set the API_KEY
tmdb.API_KEY =  login['api-key']

YEARS_TO_GET = [2000,2001]
errors = []

In [5]:
# Open saved file and preview again
basics = pd.read_csv(f"{FOLDER}title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


## Helper Functions

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [7]:
def get_movie_with_rating(movie_id):
# Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    return info

## 10. Loop Funtion to retrive YEARS with ratings

In [8]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)

    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

print(f"- Total errors: {len(errors)}")

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/211 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/244 [00:00<?, ?it/s]

- Total errors: 454


> Testing our function by looping through a list of ids: tt0848228 ("The Avengers") and tt0332280 ("The Notebook")

In [9]:
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.707,28310,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10197,PG-13


## 11. Exploratory Data Analysis

### 11.1 Contatening Data from YEARS 2000 and 2001

In [10]:
data_2000 = pd.read_csv(f"{FOLDER}final_tmdb_data_2000.csv.gz")

In [11]:
data_2001 = pd.read_csv(f"{FOLDER}final_tmdb_data_2001.csv.gz")

In [12]:
type(data_2000)

pandas.core.frame.DataFrame

In [13]:
tmdb_results_combined = pd.concat([data_2000,data_2001])

### 11.2 Save a final merged .csv.gz of all of the tmdb api data: ```tmdb_results_combined.csv.gz```

In [14]:
# Checking dataframes
tmdb_results_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2536 entries, 0 to 1308
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2536 non-null   object 
 1   adult                  2534 non-null   float64
 2   backdrop_path          1354 non-null   object 
 3   belongs_to_collection  202 non-null    object 
 4   budget                 2534 non-null   float64
 5   genres                 2534 non-null   object 
 6   homepage               169 non-null    object 
 7   id                     2534 non-null   float64
 8   original_language      2534 non-null   object 
 9   original_title         2534 non-null   object 
 10  overview               2482 non-null   object 
 11  popularity             2534 non-null   float64
 12  poster_path            2275 non-null   object 
 13  production_companies   2534 non-null   object 
 14  production_countries   2534 non-null   object 
 15  rele

In [39]:
# Filters
budget_filter = tmdb_results_combined['budget'] > 0
revenue_filter = tmdb_results_combined['revenue'] > 0

In [40]:
financial_movies_count = tmdb_results_combined[budget_filter & revenue_filter].shape[0]
print(f"There are {financial_movies_count} with relevant financial data.")

There are 354 with relevant financial data.


### 11.4 How many movies are there in each of the certification categories (G/PG/PG-13/R)?

In [41]:
financial_movies = tmdb_results_combined[budget_filter & revenue_filter]
financial_movies['certification'].value_counts()

R        142
PG-13    117
PG        30
G         13
NR         7
Name: certification, dtype: int64

### 11.5 What is the average revenue per certification category?

In [42]:
financial_movies.groupby('certification')['revenue'].agg(['mean']).style.format('${0:,.2f}')

,mean
certification,
G,"$133,216,881.69"
NR,"$22,122,566.43"
PG,"$129,080,604.10"
PG-13,"$111,073,959.08"
R,"$52,945,474.85"


### 11.6 What is the average budget per certification category?

In [43]:
financial_movies.groupby('certification')['budget'].agg(['mean']).style.format('${0:,.2f}')

,mean
certification,
G,"$44,000,000.00"
NR,"$13,528,571.43"
PG,"$51,974,907.80"
PG-13,"$46,644,422.11"
R,"$27,063,479.23"
